In [1]:
!rm -r sample_data

In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-6137c308-ef70-7623-66d0-cebed567b9ad)


In [3]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install requirements

In [4]:
!pip install SimpleITK
!pip install -r https://raw.githubusercontent.com/Lasagne/Lasagne/master/requirements.txt
!pip install https://github.com/Lasagne/Lasagne/archive/master.zip
!pip install medpy
!pip install --upgrade batchgenerators

     |████████████████████████████████| 42.5MB 72kB/s 
     | 1.1MB 9.0MB/s
  Created wheel for Lasagne: filename=Lasagne-0.2.dev1-cp36-none-any.whl size=122797 sha256=05e71c286ee34375db7eec9f473e41b3a75f6cecff669c82ab4cd0a7ce58bd33
  Stored in directory: /tmp/pip-ephem-wheel-cache-ni35os11/wheels/ca/4a/00/87f1777b229481fe76562df7c0cfb993bc88ed0cc37e3f0ed4
Successfully built Lasagne
     |████████████████████████████████| 153kB 2.9MB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp36-cp36m-linux_x86_64.whl size=753432 sha256=7a110a9a07612a23d5e482417210743441aac0f8589235f36612c1edc496e49a
  Stored in directory: /root/.cache/pip/wheels/8c/c9/9c/2c6281c7a72b9fb1570862a4f028af7ce38405008354fbf870
Successfully built medpy
     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 102kB 4.3MB/s 
  Created wheel for batchgenerators: filename=batchgenerators-0.20.1-cp36-none-any.whl size=70315 sha256=2af0e176018c2c616650563af8f808a8f797ddf254976afdd8d12

In [5]:
import batchgenerators
batchgenerators_path = batchgenerators.__file__.split("/__init")[0]

In [6]:
!wget https://raw.githubusercontent.com/MIC-DKFZ/batchgenerators/master/batchgenerators/dataloading/multi_threaded_augmenter.py -O $batchgenerators_path/dataloading/multi_threaded_augmenter.py

--2020-09-14 10:18:08--  https://raw.githubusercontent.com/MIC-DKFZ/batchgenerators/master/batchgenerators/dataloading/multi_threaded_augmenter.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11119 (11K) [text/plain]
Saving to: ‘/usr/local/lib/python3.6/dist-packages/batchgenerators/dataloading/multi_threaded_augmenter.py’

/usr/local/lib/pyth 100%[===================>]  10.86K  --.-KB/s    in 0s      

2020-09-14 10:18:08 (76.3 MB/s) - ‘/usr/local/lib/python3.6/dist-packages/batchgenerators/dataloading/multi_threaded_augmenter.py’ saved [11119/11119]



### pygpu

In [7]:
!git clone https://github.com/Theano/libgpuarray.git
%cd libgpuarray
!mkdir Build

Cloning into 'libgpuarray'...
remote: Enumerating objects: 11420, done.
remote: Total 11420 (delta 0), reused 0 (delta 0), pack-reused 11420
Receiving objects: 100% (11420/11420), 2.59 MiB | 20.70 MiB/s, done.
Resolving deltas: 100% (8379/8379), done.
/content/libgpuarray


In [8]:
%%bash
cd Build
# you can pass -DCMAKE_INSTALL_PREFIX=/path/to/somewhere to install to an alternate location
cmake .. -DCMAKE_BUILD_TYPE=Release # or Debug if you are investigating a crash
make
make install

-- The C compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Looking for strlcat
-- Looking for strlcat - not found
-- Looking for mkstemp
-- Looking for mkstemp - found
-- Found PkgConfig: /usr/bin/pkg-config (found version "0.29.1") 
-- Checking for one of the modules 'check'
-- Configuring done
-- Generating done
-- Build files have been written to: /content/libgpuarray/Build
[  1%] Generating ../../src/cluda_opencl.h.c
[  2%] Generating ../../src/cluda_cuda.h.c
Scanning dependencies of target gpuarray-static
[  3%] Building C object src/CMakeFiles/gpuarray-static.dir/cache/lru.c.o
[  5%] Building C object src/CMakeFiles/gpuarray-static.dir/cache/twoq.c.o
[  6%] Building C object src/CMakeFiles/gpuarray-static.dir/cache/disk.c.o
[  7%] Building C object sr

Tests disabled because Check was not found
/content/libgpuarray/src/gpuarray_buffer_opencl.c: In function ‘cl_memset’:
/content/libgpuarray/src/gpuarray_buffer_opencl.c:969:12: warning: ‘m’ may be used uninitialized in this function [-Wmaybe-uninitialized]
   k->refcnt--;
   ~~~~~~~~~^~
/content/libgpuarray/src/gpuarray_buffer_opencl.c:690:14: note: ‘m’ was declared here
   gpukernel *m;
              ^
/content/libgpuarray/src/gpuarray_buffer_opencl.c: In function ‘cl_memset’:
/content/libgpuarray/src/gpuarray_buffer_opencl.c:969:12: warning: ‘m’ may be used uninitialized in this function [-Wmaybe-uninitialized]
   k->refcnt--;
   ~~~~~~~~~^~
/content/libgpuarray/src/gpuarray_buffer_opencl.c:690:14: note: ‘m’ was declared here
   gpukernel *m;
              ^


In [9]:
!python setup.py build
!python setup.py install

Compiling pygpu/gpuarray.pyx because it changed.
Compiling pygpu/blas.pyx because it changed.
Compiling pygpu/_elemwise.pyx because it changed.
Compiling pygpu/collectives.pyx because it changed.
[1/4] Cythonizing pygpu/_elemwise.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/libgpuarray/pygpu/_elemwise.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/4] Cythonizing pygpu/blas.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/libgpuarray/pygpu/blas.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[3/4] Cythonizing pygpu/collectives.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, us

In [10]:
%cd ..
!rm -rf libgpuarray

/content


In [11]:
!sudo ldconfig
#or restart the environment

/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



##Import data

In [12]:
!git clone https://github.com/manigalati/ACDC2017

Cloning into 'ACDC2017'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 102 (delta 16), reused 0 (delta 0), pack-reused 65
Receiving objects: 100% (102/102), 71.92 KiB | 7.19 MiB/s, done.
Resolving deltas: 100% (50/50), done.


## Prepare dataset

In [13]:
%cd /content/ACDC2017

/content/ACDC2017


In [14]:
!mkdir -p /content/ACDC2017/train_set/ && unzip -q "/content/drive/My Drive/tesi/training.zip" -d /content/ACDC2017/train_set/

In [15]:
!unzip -q "/content/drive/My Drive/tesi/testing.zip" -d /content/ACDC2017/test_set/
!mv test_set/testing/testing test_set/tmp && rm -r test_set/testing && mv test_set/tmp test_set/testing

In [16]:
!python dataset_utils.py -i train_set/training/ -out2d train_set_2d -out3d train_set_3d

could not find dir of patient  0
could not find dir of patient  101
could not find dir of patient  102
could not find dir of patient  103
could not find dir of patient  104
could not find dir of patient  105
could not find dir of patient  106
could not find dir of patient  107
could not find dir of patient  108
could not find dir of patient  109
could not find dir of patient  110
could not find dir of patient  111
could not find dir of patient  112
could not find dir of patient  113
could not find dir of patient  114
could not find dir of patient  115
could not find dir of patient  116
could not find dir of patient  117
could not find dir of patient  118
could not find dir of patient  119
could not find dir of patient  120
could not find dir of patient  121
could not find dir of patient  122
could not find dir of patient  123
could not find dir of patient  124
could not find dir of patient  125
could not find dir of patient  126
could not find dir of patient  127
could not find dir of 

## Training

In [ ]:
!mkdir out

### Training 2D

In [ ]:
!THEANO_FLAGS='floatX=float32,device=cuda0' python run_training_2D.py -f 0 -c UNet2D_config.py

/usr/local/lib/python3.6/dist-packages/theano/gpuarray/dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7605 on context None
Mapped name None to device cuda0: Tesla P100-PCIE-16GB (0000:00:04.0)
[GpuElemwise{exp,no_inplace}(<GpuArrayType<None>(float32, vector)>), HostFromGpu(gpuarray)(GpuElemwise{exp,no_inplace}.0)]
Looping 1000 times took 0.402564 seconds
Result is [1.2317803 1.6187935 1.5227807 ... 2.2077181 2.2996776 1.623233 ]
Used the gpu


### Training 3D

In [ ]:
!THEANO_FLAGS='floatX=float32,device=cuda0' python run_training_3D.py -f 0 -c UNet3D_config.py

/content/ACDC2017/train_set_2d
/usr/local/lib/python3.6/dist-packages/batchgenerators/dataloading/data_loader.py:53: DeprecationWarning: This DataLoader will soon be removed. Migrate everything to SlimDataLoaderBase now!
  warn("This DataLoader will soon be removed. Migrate everything to SlimDataLoaderBase now!", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/batchgenerators/transforms/abstract_transforms.py:53: DeprecationWarning: This is deprecated. All applicable transfroms now have a p_per_sample argument which allows batchgenerators to do or not do an augmentation on a per-sample basis instead of the entire batch
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/batchgenerators/dataloading/data_loader.py:53: DeprecationWarning: This DataLoader will soon be removed. Migrate everything to SlimDataLoaderBase now!
  warn("This DataLoader will soon be removed. Migrate everything to SlimDataLoaderBase now!", DeprecationWarning)
Could not plot network architecture 

## Testing

In [17]:
!cp -r "/content/drive/My Drive/tesi/20180517_tmi_2018_bernard_best_RESULTS/out" /content/ACDC2017/out

### Testing 2D

In [ ]:
!THEANO_FLAGS='floatX=float32,device=cuda0' python predict_test_2D_net.py -f 0 -c /content/ACDC2017/UNet2D_config.py

/usr/local/lib/python3.6/dist-packages/theano/gpuarray/dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7605 on context None
Mapped name None to device cuda0: Tesla P100-PCIE-16GB (0000:00:04.0)
0
compiling theano functions
101
/content/ACDC2017/utils.py:30: UserWarning: The argument 'neighbors' is deprecated and will be removed in scikit-image 0.18, use 'connectivity' instead. For neighbors=8, use connectivity=3
  lbls = label(mask, 8)
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


### Testing 3D

In [31]:
!THEANO_FLAGS='floatX=float32,device=cuda0' python predict_test_3D_net.py -f 0 -c /content/ACDC2017/UNet3D_config.py

/usr/local/lib/python3.6/dist-packages/theano/gpuarray/dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7605 on context None
Mapped name None to device cuda0: Tesla P100-PCIE-16GB (0000:00:04.0)
/content/ACDC2017/train_set_3d
0
101
/content/ACDC2017/utils.py:30: UserWarning: The argument 'neighbors' is deprecated and will be removed in scikit-image 0.18, use 'connectivity' instead. For neighbors=8, use connectivity=3
  lbls = label(mask, 8)
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


### Ensemble

In [38]:
!python create_final_submission_files.py -c2d /content/ACDC2017/UNet2D_config.py -c3d /content/ACDC2017/UNet3D_config.py -o "/content/drive/My Drive/tesi/20180517_tmi_2018_bernard_best_RESULTS/resultsENSEMBLE"

/usr/local/lib/python3.6/dist-packages/theano/gpuarray/dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "
/content/ACDC2017/train_set_3d
/content/ACDC2017/out/ACDC_lasagne/UNet2D_final/test_predictions/
/content/ACDC2017/utils.py:30: UserWarning: The argument 'neighbors' is deprecated and will be removed in scikit-image 0.18, use 'connectivity' instead. For neighbors=8, use connectivity=3
  lbls = label(mask, 8)
